In [3]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import osmnx as ox
import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
import sklearn as sk
#from pyro.infer import MCMC, NUTS, Predictive, SVI, TraceMeanField_ELBO,HMC
import torch
import random
import kennard_stone as ks
import calendar
import arviz as az

In [4]:
az.style.use("arviz-doc")

In [11]:

PM25=pd.read_excel(r"\\192.168.1.59\UserFolders\michael.forsmann\Data\Copsac_Private\pm25.xlsx")
pyro.enable_validation(True)

In [12]:
#resetting  kernal parameters
pyro.clear_param_store()
#defning kernal
rbf = gp.kernels.RBF(input_dim=1)

rbf.variance = pyro.nn.PyroSample(dist.LogNormal(torch.tensor(0.01), torch.tensor(1)))

rbf.lengthscale = pyro.nn.PyroSample(dist.LogNormal(torch.tensor(0.01), torch.tensor(1)))
#data
X=torch.tensor(PM25.flow.values).float()
y=torch.tensor(PM25.pm25.values).float()

gpr = gp.models.GPRegression(X.float(),y.float(), rbf)
# define optimizer 
optimizer = torch.optim.Adam(gpr.parameters(), lr=0.005)
# define loss function
loss_fn = pyro.infer.Trace_ELBO().differentiable_loss
losses = []
locations = []
variances = []
lengthscales = []
noises = []
Abs=[]
Abst=[]
RAbs=[]
RAbst=[]
num_steps = 750
#traning
for i in range(num_steps):
    optimizer.zero_grad()
    loss = loss_fn(gpr.model, gpr.guide)
    noises.append(gpr.noise.item )
    loss.backward()
    optimizer.step()
    losses.append(loss.item())


In [16]:
plot(model=gpr, plot_observed_data=True, plot_predictions=True)


NameError: name 'plot' is not defined

In [ ]:
pyro_data = az.from_pyro(
    gpr,
    prior=prior,
    posterior_predictive=posterior_predictive,
    coords={"school": np.arange(eight_school_data["J"])},
    dims={"theta": ["school"]},
)
pyro_data

In [5]:
data = az.load_arviz_data("non_centered_eight")

In [6]:
data

Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data
	> constant_data